<a href="https://colab.research.google.com/github/helonayala/sysid/blob/main/lagrange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lagrange equations

This script presents a complete workflow for modeling and controlling mechanical system describred by their Lagrange equations.

It uses symbolic math to derive the possibly nonlinear equations of motion via Lagrange's method, linearizes the system around its upright equilibrium point, and then designs a state-feedback controller with an observer using pole placement.

TODO:
* modeling is ready (lagrange and nonlinear ss eqs)
* update Gc design and verify the CL poles
* run simulation

In [14]:
# =============================================================================
# Generalized Symbolic Modeling and Control of a Mechanical System
# =============================================================================

import sympy as sp
import numpy as np
import control as ct
import matplotlib.pyplot as plt
from IPython.display import display # For nice mathematical printing

# Use SymPy's best available renderer for equations
sp.init_printing(use_latex='mathjax')

# --- Helper Functions for Generalization ---

def create_gen_vars(ndof, t):
    """Creates symbolic generalized coordinates, velocities, and accelerations."""
    q = sp.Matrix([sp.Function(f'q{i+1}')(t) for i in range(ndof)])
    qp = q.diff(t)
    qpp = qp.diff(t)
    return q, qp, qpp

def derive_eom(T, V, q, Fext, t):
    """Derives the symbolic EOM using the Euler-Lagrange equation."""
    ndof = len(q)
    L = T - V
    qp = q.diff(t)
    qpp = qp.diff(t)
    EOM_LHS = sp.Matrix([
        (sp.diff(sp.diff(L, qp[i]), t) - sp.diff(L, q[i])).simplify()
        for i in range(ndof)
    ])
    return sp.Eq(EOM_LHS, Fext), qpp


def get_state_space_representation(EOM, q, qp, qpp, u_vec):
    """Converts the symbolic EOM into a symbolic state-space representation."""
    ndof = len(q)
    sol = sp.solve(EOM, qpp, dict=True)[0]
    q_s = sp.symbols(f'q1:{ndof+1}_s')
    qp_s = sp.symbols(f'q1:{ndof+1}p_s')

    sub_map = {}
    for i in range(ndof):
        sub_map[q[i]] = q_s[i]
        sub_map[qp[i]] = qp_s[i]
        sub_map[sp.sin(q[i])] = sp.sin(q_s[i])
        sub_map[sp.cos(q[i])] = sp.cos(q_s[i])

    x = sp.Matrix([*q_s, *qp_s])
    u = u_vec

    xp_list = list(qp_s)
    for i in range(ndof):
        xp_list.append(sol[qpp[i]].subs(sub_map))
    xp = sp.Matrix(xp_list)

    return xp, x, u

# =============================================================================
# 1. System Definition
# =============================================================================
print("--- 1. System Definition ---")
# Define symbolic variables
m, M, L, g, t, b = sp.symbols('m M L g t b')
f_in = sp.symbols('f_in')

# Define generalized coordinates, velocities, and accelerations
ndof = 2
q, qp, qpp = create_gen_vars(ndof, t)

# =============================================================================
# 2. Kinematics, Energy, and Forces for the Inverted Pendulum
# =============================================================================
print("--- 2. Kinematics, Energy, and Forces ---")
q1, q2 = q
q1p, q2p = qp

# Define kinematics
x_cart = q1
x_pend = q1 + L * sp.sin(q2)
y_pend = L * sp.cos(q2)

# Calculate velocities
v_cart_sq = x_cart.diff(t)**2
v_pend_sq = x_pend.diff(t)**2 + y_pend.diff(t)**2

# Define potential and kinetic energy
V = M * g * y_pend
T = 1/2 * m * v_cart_sq + 1/2 * M * sp.simplify(v_pend_sq)

# Define generalized forces, including friction
Fext = sp.Matrix([f_in, -b * q2p])
u_vec = sp.Matrix([f_in])

# =============================================================================
# 3. Automated EOM Derivation
# =============================================================================
print("--- 3. Automated EOM Derivation ---")
EOM, qpp = derive_eom(T, V, q, Fext, t)

# Display the symbolic Equations of Motion
print("\nDerived Equations of Motion (EOM):")
display(EOM)

# =============================================================================
# 4. Automated State-Space Formulation
# =============================================================================
print("\n--- 4. Automated State-Space Formulation ---")
xp, x, u = get_state_space_representation(EOM, q, qp, qpp, u_vec)

# Display the symbolic state-space vectors
print("\nState Vector (x):")
display(x)
print("\nInput Vector (u):")
display(u)
print("\nState Derivative Vector (xp):")
display(xp)


--- 1. System Definition ---
--- 2. Kinematics, Energy, and Forces ---
--- 3. Automated EOM Derivation ---

Derived Equations of Motion (EOM):


⎡      ⎛                          2                 2            2        ⎞    ↪
⎢      ⎜               ⎛d        ⎞                 d            d         ⎟    ↪
⎢1.0⋅M⋅⎜- L⋅sin(q₂(t))⋅⎜──(q₂(t))⎟  + L⋅cos(q₂(t))⋅───(q₂(t)) + ───(q₁(t))⎟ +  ↪
⎢      ⎜               ⎝dt       ⎠                   2            2       ⎟    ↪
⎢      ⎝                                           dt           dt        ⎠    ↪
⎢                                                                              ↪
⎢                        ⎛   2                                      2        ⎞ ↪
⎢                        ⎜  d                                      d         ⎟ ↪
⎢                1.0⋅L⋅M⋅⎜L⋅───(q₂(t)) - g⋅sin(q₂(t)) + cos(q₂(t))⋅───(q₁(t))⎟ ↪
⎢                        ⎜    2                                      2       ⎟ ↪
⎣                        ⎝  dt                                     dt        ⎠ ↪

↪        2        ⎤                 
↪       d         ⎥                 
↪ 1.0⋅m⋅───(q₁(t))⎥               


--- 4. Automated State-Space Formulation ---

State Vector (x):


⎡q₁ ₛ⎤
⎢    ⎥
⎢q₂ ₛ⎥
⎢    ⎥
⎢q1pₛ⎥
⎢    ⎥
⎣q2pₛ⎦


Input Vector (u):


[fᵢₙ]


State Derivative Vector (xp):


⎡                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                          2       2           ↪
⎢                                                         L ⋅M⋅q2pₛ ⋅sin(q₂ ₛ) ↪
⎢                                                     ──────────────────────── ↪
⎢                                                              2               ↪
⎢                                                     - L⋅M⋅cos (q₂ ₛ) + L⋅M + ↪
⎢                                                                              ↪
⎢     2  2     2                                     2                         ↪
⎢    L ⋅M ⋅q2pₛ ⋅sin(q₂ ₛ)⋅cos(q₂ ₛ)              L⋅M ⋅g⋅sin(q₂ ₛ)             ↪
⎢- ─────────────────────────

In [22]:

# =============================================================================
# 5. System Linearization
# =============================================================================
print("--- 5. System Linearization ---")
# Define equilibrium point and numerical parameters
equilibrium_point = {s: 0 for s in x}
equilibrium_point.update({s: 0 for s in u})
param_values = {m: 0.05, M: 0.5, L: 10e-2, g: 9.81, b: 0.1}

# Linearize the system
A_sym = xp.jacobian(x)
B_sym = xp.jacobian(u)
A_lin_sym = A_sym.subs(equilibrium_point)
B_lin_sym = B_sym.subs(equilibrium_point)

Alin = np.array(A_lin_sym.evalf(subs=param_values, chop=True)).astype(float)
Blin = np.array(B_lin_sym.evalf(subs=param_values, chop=True)).astype(float)

# =============================================================================
# 6. System Simplification
# =============================================================================
print("\n--- 6. System Simplification ---")
# Define the system output (pendulum angle)
output_index = 1
Clin = np.zeros((1, 2 * ndof))
Clin[0, output_index] = 1.0
Dlin = np.array([[0]])

# Get the minimal realization to find the core 2nd-order dynamics.
sys_full_ss = ct.ss(Alin, Blin, Clin, Dlin)

print("\nMinimal System Transfer Function:")
sys_min_tf = ct.minreal(ct.tf(sys_full_ss))
print(sys_min_tf)

sys_min_ss = ct.ss(sys_min_tf)
A_min, B_min, C_min, D_min = sys_min_ss.A, sys_min_ss.B, sys_min_ss.C, sys_min_ss.D


--- 5. System Linearization ---

--- 6. System Simplification ---

Minimal System Transfer Function:
2 states have been removed from the model
<TransferFunction>: sys[56]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']


       -200
------------------
s^2 + 220 s - 1079



In [25]:

# =============================================================================
# 7. Control Design and Analysis
# =============================================================================
print("\n--- 7. Control Design and Analysis ---")
# Define desired pole locations
n_poles = A_min.shape[0]
P_min = [-5,-6]     # Controller poles
Pe_min = [-10, -11]    # Observer poles

# Calculate controller and observer gains
K_min = ct.place(A_min, B_min, P_min)
Ke_min = ct.place(A_min.T, C_min.T, Pe_min).T

print(f"\nState Feedback Gain K: {K_min}")
print(f"Observer Gain Ke:\n{Ke_min}")

# Construct the compensator
At_min = A_min - Ke_min @ C_min - B_min @ K_min
Bt_min = Ke_min
Ct_min = -K_min
Dt_min = np.array([[0]])
Gc_min = ct.ss(At_min, Bt_min, Ct_min, Dt_min)

Gc = ct.tf(Gc_min)
print(f"\nCompensator Gc:\n{Gc}")

# Verify the final closed-loop poles
Gmf = ct.feedback(Gc*sys_min_tf,1)
closed_loop_poles = ct.poles(Gmf)

print("\nFinal Closed-Loop Poles:")
print(np.sort(np.round(closed_loop_poles, 4)))
print("Desired Poles were:", np.sort(P_min + Pe_min))



--- 7. Control Design and Analysis ---

State Feedback Gain K: [[-209.  1109.1]]
Observer Gain Ke:
[[-224.8455]
 [   0.995 ]]

Compensator Gc:
<TransferFunction>: sys[63]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']


-4.81e+04 s + 2.31e+05
-----------------------
s^2 - 188 s + 4.281e+04


Final Closed-Loop Poles:
[-280.1062  +0.j        4.8015  +0.j      121.6523-232.1628j
  121.6523+232.1628j]
Desired Poles were: [-11 -10  -6  -5]
